# 3. keyword 사전 활용하기

- 사용자의 질문을 vector database에 적합한 질문으로 변경해보자
- 1~4까지는 2. LangChain을 활용한 Vector Database 구성 (Pinecone + Upstage) 과 동일
- 5에서 qa_chain으로 llm에 답변 구하는 부분 제거
- 6에서 사용자의 질문을 변경하는 chain을 생성하여 기존의 qa_chain과 연결하고, llm에 질문함 

# 1. 패키지 설치

In [ ]:
%pip install python-dotenv langchain langchain-upstage==0.5.0 langchain-community langchain-text-splitters docx2txt 
%pip install langchain-pinecone
%pip install "langchain==0.1.20" "langchain-openai==0.1.8" "langchain-upstage==0.5.0" --force-reinstall

     ---------------------------------------- 1.0/1.0 MB 9.2 MB/s eta 0:00:00
  Using cached langchain_upstage-0.5.0-py3-none-any.whl (20 kB)
  Using cached pyyaml-6.0.3-cp311-cp311-win_amd64.whl (158 kB)
  Using cached sqlalchemy-2.0.44-cp311-cp311-win_amd64.whl (2.1 MB)
     -------------------------------------- 454.8/454.8 kB 9.5 MB/s eta 0:00:00
  Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
     ---------------------------------------- 2.0/2.0 MB 8.6 MB/s eta 0:00:00
     -------------------------------------- 303.1/303.1 kB 6.2 MB/s eta 0:00:00
     -------------------------------------- 311.8/311.8 kB 9.4 MB/s eta 0:00:00
     ---------------------------------------- 15.8/15.8 MB 9.6 MB/s eta 0:00:00
     -------------------------------------- 462.4/462.4 kB 9.8 MB/s eta 0:00:00
  Using cached requests-2.32.5-py3-none-any.whl (64 kB)
INFO: pip is looking at multiple versions of <Python from Requires-Python> to determine which version is compatible with other req

ERROR: Cannot install langchain-openai==0.1.8 and langchain==0.1.20 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


# 2. 지식 저장소(Knowledge Base) 구성을 위한 데이터 생성
- AI가 tax_with_markdown.docx라는 문서를 참고하여 대답할 수 있도록, <br/>
  tax_with_markdown.docx 문서 내의 내용을 가지고 Vector Database를 구성해야 합니다.<br/>
  아래 코드는 tax_with_markdown.docx를 Vector Database에 저장하기 전 적당한 크기로 나누는 작업(chunking)을 하는 코드입니다.

- [RecursiveCharacterTextSplitter](https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/)를 활용한 데이터 나누기(chunking)
    - split 된 데이터 chunk를 Large Language Model(LLM)에게 전달하면 토큰 절약 가능
    - 비용 감소와 답변 생성시간 감소의 효과
    - LangChain에서 다양한 [TextSplitter](https://python.langchain.com/v0.2/docs/how_to/#text-splitters)들을 제공

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# AI가 학습할 원본 자료를 읽는 단계(tax_with_markdown.docx 파일을 불러옵니다.)
loader = Docx2txtLoader('./tax_with_markdown.docx')

# 문서의 내용을 AI가 다루기 쉽도록 적당한 크기로 나누는 작업(chunking)을 수행합니다.
# 이 때, 어떻게 나눌지를 text_splitter를 통해 전달합니다.
# - `chunk_size` 는 split 된 chunk의 최대 크기
# - `chunk_overlap`은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

- 앞에서 잘게 나눈 문서를 실제로 AI가 이해할 수 있는 형태로 저장하는 단계 
- AI는 텍스트를 그대로 저장되거나 검색하지 않기 때문에,<br/>
잘게 나눈 문서들을 임베딩하여 저장합니다.
- 임베딩하는 방법에는 다양한 모델이 존재합니다.

In [ ]:
from langchain_upstage import UpstageEmbeddings
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv
from tqdm import tqdm

# 1. 환경변수를 불러오기
load_dotenv()

# 2. Upstage에서 제공하는 'solar-embedding-1-large' 모델을 사용
# → 문서의 내용을 숫자로 표현(벡터화)하여 유사한 의미끼리 비교할 수 있게 함
embedding = UpstageEmbeddings(model="solar-embedding-1-large")

# 3. index_name은 Pinecone 안에서 데이터를 구분하는 이름
# → 여러 개의 프로젝트나 주제를 각각의 index로 나누어 관리할 수 있음
index_name = 'tax-markdown-index'

# 4. PineconeVectorStore 인스턴스를 생성
# → 나중에 여기에 벡터화된 문서를 업로드하고 검색할 수 있음
database = PineconeVectorStore(index_name=index_name, embedding=embedding)

# Pinecone은 한 번에 너무 큰 데이터를 업로드하면 에러가 날 수 있음 (4MB 제한)
# 5. 따라서 문서를 20개씩 나누어 업로드

batch_size = 20  # 너무 크면 에러 발생, 10~50 정도가 안전

for i in tqdm(range(0, len(document_list), batch_size)):
    batch = document_list[i:i+batch_size]
    try:
        database.add_documents(batch)
    except Exception as e:
        print(f"❌ 배치 {i}~{i+batch_size} 업로드 실패: {e}")
    else:
        print(f"✅ 배치 {i}~{i+batch_size} 업로드 성공")

print("✅ 모든 문서 업로드 완료!")

# 여기까지 완료하였다면, AI가 참고할 database가 생성된 것 입니다.

c:\Users\azasu\Desktop\inflearn-llm-application\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  8%|▊         | 1/12 [00:06<01:13,  6.73s/it]

✅ 배치 0~20 업로드 성공


 17%|█▋        | 2/12 [00:10<00:51,  5.19s/it]

✅ 배치 20~40 업로드 성공


 25%|██▌       | 3/12 [00:14<00:42,  4.72s/it]

...

✅ 배치 180~200 업로드 성공


 92%|█████████▏| 11/12 [00:53<00:04,  4.78s/it]

✅ 배치 200~220 업로드 성공


100%|██████████| 12/12 [00:55<00:00,  4.62s/it]

✅ 배치 220~240 업로드 성공
✅ 모든 문서 업로드 완료!


# 3. 답변 생성을 위한 Retrieval
- `Pinecone`에 저장한 데이터를 유사도 검색(`similarity_search()`)를 활용해서 가져옴

In [ ]:
# 사용자가 궁금한 질문
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# Pinecone에 저장된 벡터 데이터에서 유사도 기반으로 관련 문서 검색
# k=2 → 가장 관련성이 높은 2개의 문서 조각을 가져옴
retrieved_docs = database.similarity_search(query, k=2)

In [ ]:
# retrieved_docs에는 검색된 문서 조각들이 리스트 형태로 저장됨
# 이 문서 조각들을 LLM에 전달하면, 질문에 맞는 답변을 생성할 수 있음
retrieved_docs

[Document(id='9f50042e-125d-4b22-87a3-871b2fba5e44', metadata={'source': './tax_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소

# 4. Augmentation을 위한 Prompt 활용
- Retrieval된 문서를 기반으로 LLM이 답변을 생성하도록 “질문과 문서를 조합하는 프롬프트(Prompt)”를 준비하는 단계
- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트(`"rlm/rag-prompt"`) 사용
- prompt: AI에게 'retrieved_docs를 참고해서 질문에 답해라'라는 지시를 전달하는 역할

In [ ]:
from langchain_upstage import ChatUpstage

# ChatUpstage: Upstage의 대화형 LLM 모델 불러오기
# → 이후 Retrieval된 문서와 결합해서 답변 생성 가능
llm = ChatUpstage() # LLM 인스턴스 생성

In [ ]:
from langchain import hub

# LangChain Hub를 통해 미리 만들어진 프롬프트를 가져오기
# → "rlm/rag-prompt"는 RAG(Retrieval-Augmented Generation)에 맞게 설계된 프롬프트
prompt = hub.pull("rlm/rag-prompt")

# 5. 답변 생성 (실제로 AI가 답변을 만드는 단계)

- LangChain에서는 RetrievalQA 라는 도구를 사용해서 지금까지 생성한 LLM + Retrieval된 문서 + Prompt를 결합할 수 있음
- [RetrievalQA](https://docs.smith.langchain.com/old/cookbook/hub-examples/retrieval-qa-chain)를 통해 LLM에 전달
    - `RetrievalQA`는 [create_retrieval_chain](https://python.langchain.com/v0.2/docs/how_to/qa_sources/#using-create_retrieval_chain)으로 대체됨
    - 실제 ChatBot 구현 시 `create_retrieval_chain`으로 변경하는 과정을 볼 수 있음

In [ ]:
from langchain.chains import RetrievalQA

# RetrievalQA: LLM + Retrieval + Prompt를 결합해 답변을 만드는 LangChain 도구
qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

C:\Users\azasu\AppData\Local\Temp\ipykernel_19504\1822491604.py:10: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query": query})


# 6. Retrieval을 위한 keyword 사전 활용

- Knowledge Base에서 사용되는 keyword를 활용하여 질문할 수 있도록 사용자 질문 수정<br/>
 ex) 사용자 질문(query)의 직장인을 거주자로 변경하는 chain 추가해보자
- LangChain Expression Language (LCEL)을 활용한 Chain 연계 <- 체인끼리 쉽게 연결하도록 해줌

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# 변환 사전 정의
# Knowledge Base에 맞게 질문을 바꾸기 위한 규칙
dictionary = ["사람을 나타내는 표현 -> 거주자"]

# prompt 생성
prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요
    사전: {dictionary}
    
    질문: {{question}}
""")

# Chain 구성
dictionary_chain = prompt | llm | StrOutputParser()

# 기존 qa_chain과 연결하여, 변환된 질문으로 답변 생성 가능
tax_chain = {"query": dictionary_chain} | qa_chain

In [ ]:
# 질문 변환
new_question = dictionary_chain.invoke({"question": query})

In [18]:
new_question

'질문: 연봉 5천만원인 거주자의 종합소득세는 얼마인가요?'

In [ ]:
# 변환된 질문으로 실제 답변 생성
ai_response = tax_chain.invoke({"question": query})

In [20]:
ai_response

{'query': '질문: 연봉 5천만원인 거주자의 종합소득세는 얼마인가요?',
 'result': '연봉 5천만원인 거주자의 종합소득세는 547만원입니다. (산출세액 547만원 - 세액공제 0만원 = 결정세액 547만원)'}